1. 已知股票现价 50 元，两个月以后可能的价格是 56 元或 48 元，无风险年利率是 10%（连续复利）。一份执行价格为 49 元、期限为 6 个月的欧式看涨期权的价格是多少？试用风险中性定价法为该期权定价。

1. 在风险中性测度下，将2个月后的股价贴现至6个月后：$ S_{T_{1}} = S_{T_{0}}*e^{r(T_1-T_0)} = (48,56) \odot e^{(0.1*\frac{1}{3})} = (49.62696545, 57.89812636)  $
   + $\odot$表示按元素相乘
2. 计算风险中性测度下的股价上涨概率：$ q = \frac{S_{0}e^{r*T}-S_{1}}{S_{2}-S_{1}} = \frac{52.56355481880121 - 49.62696545}{57.89812636-49.62696545} = 0.57044435 $
   + 其中$S_{0}$表示变化前的股价，$S_{1}$,$S_{2}$分别表示下跌和上涨的股价
   + 由此可得风险中性概率测度：{0.57044435, 0.42955565}
3. 离散状态下的风险中性定价方法：$ V_{0} = e^{-r*\Delta t}(1-q,q)\cdot(S(1),V(2))^{T} = 0.951229424500714*5.345202476061453 = 5.0845138751437275 $
   

In [16]:
from numpy import e,array,dot,where
S_t1 = array([48,56])*e**(0.1*1/3)
S = 50
K = 49
q = array([1-((50*e**(0.1*1/2)-48)/(56-48)),(50*e**(0.1*1/2)-48)/(56-48)])
X_t1 = where(S_t1-K>0,S_t1-K,0)
V = e**(-0.1*1/2)*dot(X_t1,q)
print(f'期权价格：{V}')

期权价格：5.0845138751437275


2. 假设某不付红利股票价格遵循几何布朗运动，其预期年收益率为 16% ，年波动率为 30% ，该股票当天收盘价为 50 元，求
   1. 第二天收盘时的预期价格；
   2. 第二天收盘时股价的标准差；
   3. 在置信度为 95%的情况下，该股票第二天收盘时的价格范围。
   4. 以该股票为标的资产，无风险利率为 10%，请问 3 个月期的、执行价格等于 50 元的欧式看跌期权价格等于多少？

1. $ E_{P}(S_{1}) = E_{P}(S_{0}*e^{\mu(\Delta t)+\sigma W_{\Delta t}}) = S_{0}*e^{\mu\Delta t+\frac{1}{2}\sigma^{2}\Delta t} = 50*e^{0.16\frac{1}{365}+\frac{1}{2}{0.3}^{2}\frac{1}{365}} = 50.028090079352374 $

2. $  \sqrt{Var(S_{1})} = \sqrt{E_{P}(S_{1}^{2})- E_{P}(S_1)^{2}} = \sqrt{E_{P}(S_{0}^{2}*e^{2*\mu\Delta t+2*\sigma W_{\Delta t}}) - S_{0}^{2}*e^{2*\mu(\Delta t)+\sigma^{2}\Delta t}} = S_{0}*e^{\mu\Delta t}\sqrt{e^{2*\sigma^{2}\Delta t}-e^{\sigma^{2}\Delta t}} =   50*e^{0.16\frac{1}{365}}\sqrt{e^{2*{0.3}^{2}\frac{1}{365}}-e^{{0.3}^{2}\frac{1}{365}}} = 0.7856254029367408 $

3. 95%置信区间为：$[-1.96*\sqrt{Var(S_{1})}+ E_{P}(S_{1}),1.96*\sqrt{Var(S_{1})}+ E_{P}(S_{1})] = [48.488264289596366, 51.56791586910838] $

4. $p^{E}_{t} = K*e^{-r(T-t)}*\Phi(-d_{2}) - S_{t}*\Phi(-d_{1}) = 2.3759406675006467 $
   + 其中$d_2 = \frac{ln(\frac{S_{t}}{K})+(r-\frac{1}{2}\sigma^{2})}{\sigma\sqrt(T-t)} = 0.0917$,$d_1 = d_2 + \sigma\sqrt{T-t}$

In [27]:
from math import e,sqrt,log
from statistics import NormalDist
E_S1 = 50*e**(1/365*(0.16+0.5*0.3**2))
print(f'预期股价：{E_S1}')
std_dev = 50*e**(1/365*0.16)*sqrt(e**(2*1/365*0.3**2)-e**(1/365*0.3**2))
print(f'预期标准差：{std_dev}')
conf_intv = [-1.96*std_dev+E_S1,1.96*std_dev+E_S1]
print(f'预期置信区间：{conf_intv}')
d2 = (log(50/50)+(0.1-0.5*0.3**2)*(0.25))/(0.3*sqrt(0.25))
pe_t = -50*NormalDist(0,1).cdf(-d2-0.3*sqrt(0.25))+50*e**(-0.1*0.25)*NormalDist(0,1).cdf(-d2)
print(f'预期看跌期权：{pe_t}')

预期股价：50.028090079352374
预期标准差：0.7856254029367408
预期置信区间：[48.488264289596366, 51.56791586910838]
预期看跌期权：2.3759406675006467


3. 请在充分理解 BSM 期权定价公式推导的基础上完成以下练习：
    1. 证明$𝑵(𝒅𝟏) = \frac{\partial 𝒄}{\partial 𝑺}$；
    2. 证明在风险中性世界中，欧式看跌期权被执行的概率是𝑵(−𝒅𝟐)；
    3. 一只或有现金期权满足：若到期时标的资产价格大于执行价格则回报$100，反之则没有回报，试为该或有现金期权定价。

1. 已知$d_1 = d_2+\sigma\sqrt{T-t}$,$ S_{t}*\phi(d_{1}) = K * e^{-r(T-t)} *\phi(d_{2})$
   
    \begin{aligned}
      \frac{\partial c}{\partial S} &= \frac{\partial( S_{t}*\Phi(d_{1}) - K * e^{-r(T-t)} *\Phi(d_{2}))}{\partial S} \\
      &= \Phi(d_{1}) + S_{t}*\phi(d_{1}) *\frac{\partial d_1}{\partial S} - K * e^{-r(T-t)} *\phi(d_{2}) *\frac{\partial (d_1-\sigma\sqrt{T-t})}{\partial S} \\
      &= \Phi(d_{1}) +0 = \Phi(d_{1})
   \end{aligned}

2. 当$S_{T}<K$时，欧式看跌期权会行权

    \begin{aligned}
      S_{T}-K = S_{t}*e^{\mu(T-t)+\sigma(z_{T}-z_{t})}-K &<0 \\
      S_{t}*e^{(r-\frac{1}{2}\sigma^{2})(T-t)+\sigma(\tilde{z}_{T} - \tilde{z}_{t})}-K &<0 \\
      \frac{\tilde{z}_{T} - \tilde{z}_{t}}{\sqrt{T-t}} < \frac{ln(\frac{S_{t}}{K})+(r-\frac{1}{2}\sigma^{2})(T-t)}{\sigma\sqrt{T-t}} = -d_2
   \end{aligned}

   其中$\tilde{z}_{t} = z_{t} + \frac{\mu-r+\frac{1}{2}\sigma^{2}}{\sigma}$

   显然，$\frac{\tilde{z}_{T} - \tilde{z}_{t}}{\sqrt{T-t}}$为Q测度下维纳过程，故$P_{Q}(S_{T}-K<0) = P_{Q}(\frac{\tilde{z}_{T} - \tilde{z}_{t}}{\sqrt{T-t}}<-d_2) = N(-d_2) $

3. 已知$X_{T} = 100,if \space S_{T}>K$ 
   
   所以 $c^{digital}_{t} = e^{-r(T-t)} * 100 * P_{Q}(S_{T}>K) = 100* e^{-r(T-t)} *N(d_2)$
   

4. 证明题
    + （1）$S_{t}$表示标的资产价格，证明$𝒆^{S_{t}}$一定不是该资产某个衍生品的价格$V_{t}$，而$\frac{e^{(\sigma^{2}-2r)(T-t)}}{S_{t}}$一定是。
    + （2）假设r 是在T 时刻支付 1 美元的零息票债券按连续复利计息的到期收益率。r 遵循如下过程$$ dr = a(x_{0}-r)dt + \sigma r dz $$其中a,r , 和$\sigma$是正常数，dz 是维纳过程。
      + 请写出债券价格遵循的过程。

1. 设$f(S_{t},t) = e^{S_{t}}$,$g(S_{t},t) =S_{t}^{-1}*e^{(\sigma^{2}-2r)(T-t)} $
   + 由伊藤引理得：$df = 0 +  e^{S_{t}}(dS_{t} + d(S_{t})^{2}) = e^{S_{t}}(dS_{t} + dt)$，将$df$转换为鞅时，会消去$dS_t$项，因此，$f(S_{t},t)$不可能是衍生品价格
   + 由伊藤引理可得：$dg = S_{t}^{-1}*e^{(\sigma^{2}-2r)(T-t)}[(2r-\sigma^{2}+ S_{t}^{-2})dt -S_{t}^{-1}dS_{t}] = -S_{t}^{-2}*e^{(\sigma^{2}-2r)(T-t)}dS_{t} $,可见$dg$是鞅，因此一定是衍生品价格

2. 因为债券价格$B_{t} = e^{-rt}$

   由伊藤引理可得：

    \begin{aligned}
      dB_t &= \frac{\partial B_{t} }{\partial t}dt + \frac{\partial B_{t} }{\partial r}dr + \frac{\partial^{2} B_{t} }{\partial r^{2}} (dr)^{2}   \\
      & = B_{t}(-r dt -t dr + t^{2}(dr)^{2}) \\
      & =  e^{-rt}[(t^{2}\sigma^{2}r^{2}-r) dt -t (a(x_{0}-r)dt + \sigma r dz)]   \\
      & =  e^{-rt}[(t^{2}\sigma^{2}r^{2}-r - a t x_{0}+atr) dt+ \sigma r dz]    
   \end{aligned}